# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [6]:
from train import clean_data
from azureml.core import Workspace, Experiment
from azureml.train.automl import AutoMLConfig
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [25]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'capstone-experiment'

experiment=Experiment(ws, experiment_name)

# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: Must upload the dataset from local files and name it loan-dataset
found = False
key = "loan-dataset"
description_text = "Dataset of credit risk"

if key in ws.datasets.keys(): 
    found = True
    ds = ws.datasets[key]
else:
    assert key=="loan-dataset",'Need to upload dataset to Azure UI'

df = ds.to_pandas_dataframe()

# Use the clean_data function from the train.py file to clean your data.
x, y = clean_data(df)

# All data must be merged into a single dataframe for Automlconfig
df_all = x.merge(y,left_index=True,right_index=True)

# Convert the cleaned data back to an Azure dataset
datastore = ws.get_default_datastore()
ds = Dataset.Tabular.register_pandas_dataframe(
        dataframe=df_all, 
        name='loan-dataset-automl', 
        description='loan dataset for project',
        target=datastore
    )

Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/ed829126-106e-4ca0-bc7e-50cb8a4a1225/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


In [7]:
# Setting up compute as Standard_D2_V2
cluster_name = "compute-project"

## Used some code from train-hyperparameter-tune-deploy-with-keras
# choose a name for your cluster

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2', 
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

# can poll for a minimum number of nodes and for a specific timeout. 
# if no min node count is provided it uses the scale settings for the cluster
compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

Found existing compute target
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2023-03-21T15:58:56.886000+00:00', 'errors': None, 'creationTime': '2023-03-21T13:28:47.502353+00:00', 'modifiedTime': '2023-03-21T13:28:54.004308+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [41]:
# TODO: Put your automl settings here
automl_settings = {}

automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=df_all,
                             label_column_name="Target",   
                             enable_early_stopping= True,
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [45]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}

automl_config = AutoMLConfig(
    compute_target=compute_target,
    task='classification',
    training_data=df_all,
    label_column_name='Target',
    n_cross_validations=5,
    **automl_settings
)

In [ ]:
# TODO: Submit your experiment
from azureml.core.dataset import Dataset
# Dataset.Tabular.register_pandas_dataframe(df_all,'loan-dataset-automl','a description')
# remote_run = experiment.submit(automl_config)
automl_run = experiment.submit(config=automl_config)

2023-03-21:20:45:03,482 WARNING  [local_experiment_driver.py:147] Running on local machine. Note that local runs always run synchronously even if you use the parameter 'show_output=False'


Experiment,Id,Type,Status,Details Page,Docs Page
capstone-experiment,AutoML_58ec7730-50bd-4bb8-a0fb-00e76b9bef6c,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
